In [1]:
import pandas as pd

import logging
import logging.config
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder

import ast

In [2]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# cargar datos para conectar a AGROCITTCA
data = open(ruta + '/_keys/agrocittca.data','r')
password = data.readlines()[0]
data.close()

# extraccion de los datos
datos_cnx = ast.literal_eval(password)

# lectura de llave ssh
mypkey = paramiko.RSAKey.from_private_key_file(ruta + '/_keys/llave_db_agrocittca', password='micontrasena')
# ssh variables
ssh_host = datos_cnx['ssh_host']
ssh_port = datos_cnx['ssh_port']
localhost = datos_cnx['localhost']
localport = datos_cnx['localport']
ssh_user = datos_cnx['ssh_user']
# database variables
sql_username = datos_cnx['sql_username']
sql_password = datos_cnx['sql_password']
sql_main_database = datos_cnx['sql_main_database']

In [3]:
# funcon para ejecutar una script SQL
def dbQuery(q):
    """ access the database over the SSH tunnel and execute the query """
    logger = logging.getLogger()
    logger.debug("Connecting to reporting server...", end="")
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(localhost, 3306)) as tunnel:
        logger.debug("Connected")
        logger.debug("Connecting to database...")
        try:
            conn = pymysql.connect(
                host='127.0.0.1', 
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor)
            logger.debug("Connected")
            logger.debug("Executing query...")
            cursor = conn.cursor()
            cursor.execute(q)
            data = cursor.fetchall()
            logger.debug("Done")
        finally:
            conn.close()
    return data

In [4]:
# SELECCIONAR TODOS LOS USUARIOS CAÑEROS ACTIVOS
#select codigo, name, apellido_paterno, apellido_materno, username from users where estado='activo' and tipo = 'propietario
cantidad_usuarios = dbQuery('select count(*) cantidad_usuarios from users where estado=\'activo\' and tipo = \'propietario\'')[0]['cantidad_usuarios']
cantidad_props = dbQuery('select count(*) cantidad_props from propiedad prop\
                            inner join periodo per on prop.periodo_id=per.id\
                            where per.nombre=\'CAMPAÑA 2023\' and prop.estado=\'activo\'')[0]['cantidad_props']
 

In [5]:
cantidad_usuarios

1388

In [6]:
cantidad_props

156

## USUARIOS REGISTRADOS

In [7]:
usuarios_reg = dbQuery('select name, apellido_paterno, apellido_materno, username, codigo from users where estado=\'activo\' and tipo=\'propietario\'')

In [8]:
lista_users = pd.DataFrame(usuarios_reg)

In [9]:
lista_users.head(3)

,name,apellido_paterno,apellido_materno,username,codigo
0,12504 - LAUREANA MARTINEZ,MARTINEZ,DE SALO,12504,12504
1,2641 - ROBERTO CRONENBOLD,CRONENBOLD,AGUILERA,rcronenbold,2641
2,2816 - ALBERTO DONOSO,DONOSO,BLANCO,adonoso,2816


In [10]:
lista_users.to_excel('lista_usuarios.xlsx')

## USUARIOS ACTIVOS

In [11]:
usuarios_act = dbQuery('select tok.created_at, usu.codigo, usu.name, usu.apellido_paterno, usu.apellido_materno, username from oauth_access_tokens tok\
                        inner join users usu on tok.user_id=usu.id')

In [12]:
lista_users_act = pd.DataFrame(usuarios_act)

In [13]:
lista_users_act.to_excel('lista_usuarios_act.xlsx')

# ALERTAS

In [18]:
alertas = dbQuery('select * from alerta where created_at > \'2023-03-01\' and comentario <> \'NAX\'')

In [19]:
lista_alertas = pd.DataFrame(alertas)

In [21]:
lista_alertas.to_excel('alertas.xlsx')

## ACTUALIZAR NOMBRES DE LABORES EN MI PRECIO

In [7]:
# al modificar los nombres de las actividades desde el admin, 
# los usuarios que ya tengan actividades con precios personalizados no se actualizan los nombres
# este script actualiza los nombres de actividades 'mi_precio' con los nombres se actividades globales
query = 'update tipo_actividad_mi_precio mi set mi.nombre=(select nombre from tipo_actividad ac where ac.id=mi.tipo_actividad_id)'
res = dbQuery(query)

In [26]:
res

()

In [9]:
# modifica los nombres de la lista de productos con precio personalizado 
# esto en caso de que el nombre del producto estandar cambie de nombre
query = 'update producto_act_miprecio pam set pam.nombre=(select p_a.nombre from producto_actividad p_a where p_a.id=pam.producto_actividad_id)'
res = dbQuery(query)